In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [27]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain_openai import ChatOpenAI
from langchain.globals import set_debug

In [16]:
llm = ChatOpenAI(temperature=0)

In [17]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm)

In [18]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, # LLM might output something that can't be parsed. Pass it back to the LLM and have it correct when this is true.
    verbose=True
)

In [20]:
agent("If my mortgage is $2000 a month and I pay 5% property tax per year on a 500,000 home, how much do I pay in a year total?")



> Entering new AgentExecutor chain...
Thought: To calculate the total amount paid in a year, we need to find the annual property tax amount first and then add it to the total mortgage amount for the year.

Action:
```
{
  "action": "Calculator",
  "action_input": "500000 * 0.05"
}
```
Observation: Answer: 25000.0
Thought:```
{
  "action": "Calculator",
  "action_input": "2000 * 12 + 25000"
}
```
Observation: Answer: 49000
Thought:Final Answer: $49,000

> Finished chain.


{'input': 'If my mortgage is $2000 a month and I pay 5% property tax per year on a 500,000 home, how much do I pay in a year total?',
 'output': '$49,000'}

In [23]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [24]:
customer_list = [
    ["Harrison", "Chase"], 
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"], 
    ["Geoff","Fusion"], 
    ["Trance","Former"],
    ["Jen","Ayai"]
]

In [28]:
set_debug(True)
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
set_debug(False)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

In [29]:
from langchain.agents import tool
from datetime import date

In [30]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [31]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [32]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I can use the `time` tool to find out today's date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2024-04-16
Thought:Could not parse LLM output: Final Answer
Observation: Invalid or incomplete response
Thought:It seems there was an issue with the output formatting. Let's try using the `time` tool again to get today's date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
``` 


Observation: 2024-04-16
Thought:Final Answer: 2024-04-16

> Finished chain.
